In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib
matplotlib.rcParams['figure.figsize'] = [15, 10]

In [1]:
import datetime
import os
import matplotlib.pyplot as plt
import numpy as np
import torch
import minerl
from tqdm import tqdm

from torch import nn, optim
import torch.nn.functional as F
from torch.optim.lr_scheduler import ReduceLROnPlateau
from utilities import flatten, unflatten, to_batch_shape, to_torch_channels
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader

import autoencoder
from AdvancedAutoencoder import AdvancedAutoencoder
from networks import WMAutoencoder, WM_VAE, VisionModel

from dataset_preprocessing import MineRlSequenceDataset


/home/marthinwurer/.pyenv/versions/3.9.6/envs/minerl/lib/python3.9/site-packages/gym/logger.py:34: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize("%s: %s" % ("WARN", msg % args), "yellow"))


In [2]:
BATCH_SIZE = 256
LEARNING_RATE = 0.0001

In [3]:
dataset = MineRlSequenceDataset("data/npy_obtain_diamond", 1)

In [4]:
train_dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=8)

In [5]:
train_features = next(iter(train_dataloader))
train_features[0].shape

torch.Size([256, 1, 3, 64, 64])

In [6]:
model = VisionModel(256)

In [7]:
povs = train_features[0]
batch_size, timesteps, C, H, W = povs.shape
input_data = povs.view(timesteps * batch_size, C, H, W)
input_data.shape

torch.Size([256, 3, 64, 64])

In [8]:
with torch.no_grad():
    outs = model(input_data)

In [9]:
loss_func = autoencoder.abs_loss

In [10]:
loss_func(outs[0], input_data, outs[1])

tensor(0.3287)

In [11]:
model.cuda()

VisionModel(
  (encoder): VisionEncoder(
    (encoder): Sequential(
      (0): CoordConv2d(
        3, 32, kernel_size=(4, 4), stride=(2, 2)
        (addcoords): AddCoords()
        (conv): Conv2d(5, 32, kernel_size=(4, 4), stride=(2, 2))
      )
      (1): SiLU()
      (2): SkippableLayerNorm()
      (3): CoordConv2d(
        32, 64, kernel_size=(4, 4), stride=(2, 2)
        (addcoords): AddCoords()
        (conv): Conv2d(34, 64, kernel_size=(4, 4), stride=(2, 2))
      )
      (4): SiLU()
      (5): SkippableLayerNorm()
      (6): CoordConv2d(
        64, 128, kernel_size=(4, 4), stride=(2, 2)
        (addcoords): AddCoords()
        (conv): Conv2d(66, 128, kernel_size=(4, 4), stride=(2, 2))
      )
      (7): SiLU()
      (8): SkippableLayerNorm()
      (9): CoordConv2d(
        128, 256, kernel_size=(4, 4), stride=(2, 2)
        (addcoords): AddCoords()
        (conv): Conv2d(130, 256, kernel_size=(4, 4), stride=(2, 2))
      )
      (10): SiLU()
      (11): SkippableLayerNorm()
  

In [19]:
losses = []

In [ ]:
with tqdm(enumerate(train_dataloader, 0), total=len(dataset)/(BATCH_SIZE), unit="batch") as t:
    for i, data in t:
        # unpack the data and send it to the gpu
        povs, _, _, _, _, _ = data
        batch_size, timesteps, C, H, W = povs.shape
        images = povs.view(timesteps * batch_size, C, H, W).cuda()
        
        
        